# 7B_cmap_enrichment

for the egenes we get for mpra see if the cmap egenes, per disease are enriched 

this  python part is to process the gmt file

In [30]:
# !pip install mygene

In [31]:
import pandas as pd
import os, glob, re
import gzip
from collections import Counter, defaultdict
import mygene

mg = mygene.MyGeneInfo()


In [91]:
def write_gmt(set_to_genes_dict, save_file):
    with open(save_file, 'w') as f:
        for setname, genes in set_to_genes_dict.items():
            line = [setname, 'NA'] + sorted([x for x in genes if x!=''])
            f.write('\t'.join(line) + '\n')
            
def read_gmt(gmt_filepath):
    # read to pandas df
    if gmt_filepath.endswith('gz'):
        f=gzip.open(gmt_filepath,'r')
    else:
        f=open(gmt_filepath, 'rb')
    file_content=f.readlines()
    set_to_genes_dict = {}
    for idx in range(len(file_content)):
        line_arr = file_content[idx].decode("utf-8") .strip().split('\t')
        setname = line_arr[0]
        genes_arr = line_arr[2:]
        genes_str = '|'.join(genes_arr)
        num_genes = len(genes_arr)
        set_to_genes_dict[idx] = {'setname':setname,'genes_str':genes_str, 'genes_arr':genes_arr,'num_genes':num_genes}
    f.close()
    return pd.DataFrame.from_dict(set_to_genes_dict,orient='index')

def get_entrez_id_dict(gene_list):
    out = mg.querymany(gene_list, scopes='symbol', fields='entrezgene', species='human')
    out_entrez = []
    for dict_res in out:
        out_entrez.append(str(dict_res.get('entrezgene','')))
    return dict(zip(gene_list, out_entrez))

In [46]:
# glob.glob('/Users/mguo123/Google Drive/0_altman/db/cmap/*')

# 1. make gmt of all the repurposed drug categories with actual drug targets

In [62]:
# takes a bit
repurposing_drugs_df = pd.read_csv('/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_20200324.txt',sep='\t',header=9)
repurposing_drugs_df = repurposing_drugs_df[~repurposing_drugs_df.disease_area.isna()]
repurposing_drugs_df = repurposing_drugs_df[~repurposing_drugs_df.target.isna()]
repurposing_drugs_df['target_arr'] = repurposing_drugs_df.target.str.split('|')


In [63]:
# get all entrez
all_genes_repurposed = sorted(set('|'.join(repurposing_drugs_df.target.values).split('|'))) 
gene_to_entrez_repurposed_dict = get_entrez_id_dict(all_genes_repurposed)

# repurposing_drugs_df['target_entrez_arr'] = repurposing_drugs_df['target_arr'].apply(lambda x: get_entrez_id(x))


querying 1-1000...done.
querying 1001-1153...done.
Finished.
5 input query terms found dup hits:
	[('DDR1', 2), ('IGLC1', 2), ('PSMB1', 2), ('SLC5A8', 2), ('SLC6A3', 2)]
17 input query terms found no hit:
	['ALPPL2', 'ATP5A1', 'ATP5B', 'ATP5C1', 'ATP5D', 'CARS', 'CDO-1', 'GPR97', 'GUCY1A3', 'GUCY1B3', 'HR
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [77]:
repurposing_drugs_df['target_entrez_arr'] = repurposing_drugs_df.target_arr.apply(lambda arr: [gene_to_entrez_repurposed_dict[x] for x in arr])
repurposing_drugs_df['target_entrez'] = repurposing_drugs_df['target_entrez_arr'].apply(lambda x: '|'.join(x).strip('|'))

In [78]:
repurposing_drugs_df[:5]

,pert_iname,clinical_phase,moa,target,disease_area,indication,target_arr,target_entrez_arr,target_entrez
0,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,ADRA2A|ADRA2B|ADRA2C|CALY|DRD1|DRD2|DRD3|DRD4|...,neurology/psychiatry,Parkinson's Disease,"[ADRA2A, ADRA2B, ADRA2C, CALY, DRD1, DRD2, DRD...","[150, 151, 152, 50632, 1806, 1812, 1813, 1814,...",150|151|152|50632|1806|1812|1813|1814|1815|331...
34,abamectin,Launched,benzodiazepine receptor agonist,GABBR1|GABBR2,infectious disease,gastrointestinal parasites,"[GABBR1, GABBR2]","[2548, 2550]",2548|2550
39,abemaciclib,Launched,CDK inhibitor,CDK4|CDK6,oncology,breast cancer,"[CDK4, CDK6]","[1019, 1021]",1019|1021
40,abiraterone,Launched,androgen biosynthesis inhibitor,CYP11B1|CYP17A1,oncology,prostate cancer,"[CYP11B1, CYP17A1]","[1584, 1586]",1584|1586
41,abiraterone-acetate,Launched,androgen biosynthesis inhibitor,CYP17A1,oncology,prostate cancer,[CYP17A1],[1586],1586


In [79]:
repurposing_drugs_df.pert_iname.sort_values()

0                   (R)-(-)-apomorphine
6639    17-hydroxyprogesterone-caproate
6725               4-aminohippuric-acid
6755              5-aminolevulinic-acid
6760                     5-fluorouracil
                     ...               
6596                       zolmitriptan
6600                         zonisamide
6603                        zotarolimus
6604                           zotepine
6608                     zuclopenthixol
Name: pert_iname, Length: 1478, dtype: object

### A. make gmt with the drugs themselves

In [164]:
drug_to_genes_dict = pd.Series(repurposing_drugs_df.target_entrez_arr.values, index=repurposing_drugs_df.pert_iname.values).to_dict()
save_repurpose_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_target.gmt'
write_gmt(drug_to_genes_dict, save_repurpose_gmt)
read_gmt(save_repurpose_gmt)

,setname,genes_str,genes_arr,num_genes
0,(R)-(-)-apomorphine,150|151|152|1806|1812|1813|1814|1815|3315|3350...,"[150, 151, 152, 1806, 1812, 1813, 1814, 1815, ...",16
1,abamectin,2548|2550,"[2548, 2550]",2
2,abemaciclib,1019|1021,"[1019, 1021]",2
3,abiraterone,1584|1586,"[1584, 1586]",2
4,abiraterone-acetate,1586,[1586],1
...,...,...,...,...
1473,17-hydroxyprogesterone-caproate,190|2066|2099|367|4835|8566|9970,"[190, 2066, 2099, 367, 4835, 8566, 9970]",7
1474,4-aminohippuric-acid,6506,[6506],1
1475,5-aminolevulinic-acid,210,[210],1
1476,5-fluorouracil,1803|51175,"[1803, 51175]",2


### B. make gmt with drug categories

In [165]:
cat_counts = repurposing_drugs_df.disease_area.value_counts()[repurposing_drugs_df.disease_area.value_counts()>10]
display(cat_counts)
drug_categories = list(cat_counts.index)
print(drug_categories)

neurology/psychiatry      274
cardiology                167
infectious disease        122
endocrinology             104
oncology                   97
gastroenterology           87
dermatology                60
pulmonary                  58
hematologic malignancy     48
ophthalmology              45
rheumatology               43
hematology                 27
allergy                    25
urology                    21
obstetrics/gynecology      15
Name: disease_area, dtype: int64

['neurology/psychiatry', 'cardiology', 'infectious disease', 'endocrinology', 'oncology', 'gastroenterology', 'dermatology', 'pulmonary', 'hematologic malignancy', 'ophthalmology', 'rheumatology', 'hematology', 'allergy', 'urology', 'obstetrics/gynecology']


In [166]:
cat_to_genelist = defaultdict(set)
for idx, row in repurposing_drugs_df.iterrows():
    genes = row['target_entrez_arr']
    for cat in drug_categories:
        if cat in row['disease_area'].split('|'):
            cat_to_genelist[cat] = cat_to_genelist[cat].union(genes)
        

In [167]:
save_repurpose_gmt_cat = '/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_target_cat.gmt'
write_gmt(cat_to_genelist, save_repurpose_gmt_cat)
read_gmt(save_repurpose_gmt_cat)

,setname,genes_str,genes_arr,num_genes
0,neurology/psychiatry,10019|10038|10242|10352|10368|10369|10599|1066...,"[10019, 10038, 10242, 10352, 10368, 10369, 105...",481
1,infectious disease,100|10368|10369|10376|10381|10382|10383|10599|...,"[100, 10368, 10369, 10376, 10381, 10382, 10383...",234
2,oncology,10188|1019|1021|10376|10381|10382|10383|10988|...,"[10188, 1019, 1021, 10376, 10381, 10382, 10383...",160
3,hematologic malignancy,100|10013|10014|1017|10188|10376|10381|10382|1...,"[100, 10013, 10014, 1017, 10188, 10376, 10381,...",163
4,endocrinology,10019|10060|10376|10682|1080|10901|11016|11271...,"[10019, 10060, 10376, 10682, 1080, 10901, 1101...",266
5,cardiology,100|10060|10188|10280|10449|1066|1080|10846|10...,"[100, 10060, 10188, 10280, 10449, 1066, 1080, ...",307
6,ophthalmology,10800|11238|11251|1128|1129|1131|1132|1133|113...,"[10800, 11238, 11251, 1128, 1129, 1131, 1132, ...",73
7,rheumatology,100126572|10105|10280|1036|10376|10560|1080|10...,"[100126572, 10105, 10280, 1036, 10376, 10560, ...",181
8,pulmonary,10019|10682|1080|10800|11016|11251|1128|1129|1...,"[10019, 10682, 1080, 10800, 11016, 11251, 1128...",135
9,hematology,10599|1080|11255|1128|1131|142|1429|146|147|14...,"[10599, 1080, 11255, 1128, 1131, 142, 1429, 14...",85


### make gmt with psych drug classes
read drug specific info

In [168]:
# psych drug info
drugs_df = pd.read_csv('/Users/mguo123/Google Drive/1_khavari/noncancer_project/cmap/psych_drugs.tsv',sep='\t')
print(drugs_df.shape)
drugs_df[:5]

(166, 10)


,Full_ATC,Drug_name,ATC_level3,ATC_level3_name,ATC_level4,ATC_level4_name,Manual Annotation,Disease,Paired with,Comments
0,N05AA01,chlorpromazine,N05A,Antipsychotics,N05AA,Phenothiazines with aliphatic side-chain,Typical antipsychotic,"SCZ, BPD",NaN,First antipsychotic
1,N05AA02,levomepromazine,N05A,Antipsychotics,N05AA,Phenothiazines with aliphatic side-chain,Phenothiazine neuroleptic,"SCZ, BPD",NaN,"Lots of off-target effects, “dirty drug”"
2,N05AA03,promazine,N05A,Antipsychotics,N05AA,Phenothiazines with aliphatic side-chain,Phenothiazine,SCZ,"olanzapine, quetiapine","Older medication, discontinued"
3,N05AA04,acepromazine,N05A,Antipsychotics,N05AA,Phenothiazines with aliphatic side-chain,Phenothiazine,NaN,NaN,"Mostly animal use for sedation, chlorpromazine..."
4,N05AA05,triflupromazine,N05A,Antipsychotics,N05AA,Phenothiazines with aliphatic side-chain,Phenothiazine,NaN,NaN,"Too many side effects, rarely used"


In [169]:
repurposing_drugs_df_psych = repurposing_drugs_df[repurposing_drugs_df.pert_iname.isin(drugs_df.Drug_name.values)]
repurposing_drugs_df_psych = repurposing_drugs_df_psych.merge(drugs_df, how='left',left_on='pert_iname',right_on='Drug_name')
print(repurposing_drugs_df_psych.shape)
repurposing_drugs_df_psych[:5]

(91, 19)


,pert_iname,clinical_phase,moa,target,disease_area,indication,target_arr,target_entrez_arr,target_entrez,Full_ATC,Drug_name,ATC_level3,ATC_level3_name,ATC_level4,ATC_level4_name,Manual Annotation,Disease,Paired with,Comments
0,acepromazine,Launched,dopamine receptor antagonist,ADRA1A|ADRA1B|DRD1|DRD2|HTR1A|HTR2A,neurology/psychiatry,sedative,"[ADRA1A, ADRA1B, DRD1, DRD2, HTR1A, HTR2A]","[148, 147, 1806, 1812, 3315, 3355]",148|147|1806|1812|3315|3355,N05AA04,acepromazine,N05A,Antipsychotics,N05AA,Phenothiazines with aliphatic side-chain,Phenothiazine,NaN,NaN,"Mostly animal use for sedation, chlorpromazine..."
1,acetophenazine,Launched,dopamine receptor antagonist,DRD1|DRD2,neurology/psychiatry,psychosis,"[DRD1, DRD2]","[1806, 1812]",1806|1812,N05AB07,acetophenazine,N05A,Antipsychotics,N05AB,Phenothiazines with piperazine structure,Typical antipsychotic,NaN,NaN,NaN
2,agomelatine,Launched,melatonin receptor agonist|serotonin receptor ...,HTR2A|HTR2B|HTR2C|MTNR1A|MTNR1B,neurology/psychiatry,depression,"[HTR2A, HTR2B, HTR2C, MTNR1A, MTNR1B]","[3355, 3356, 3357, , ]",3355|3356|3357,N06AX22,agomelatine,N06A,Antidepressants,N06AX,Other antidepressants,Atypical antidepressant,MDD,NaN,NaN
3,amisulpride,Launched,dopamine receptor antagonist,DRD2,neurology/psychiatry,psychosis|bipolar disorder|schizophrenia,[DRD2],[1812],1812,N05AL05,amisulpride,N05A,Antipsychotics,N05AL,Benzamides,Atypical antipsychotic,"SCZ, BPD",NaN,Dopamine antagonist
4,amitriptyline,Launched,norepinephrine inhibitor|norepinephrine reupta...,ADRA1A|ADRA1B|ADRA1D|ADRA2A|ADRB1|ADRB2|ADRB3|...,neurology/psychiatry,depression,"[ADRA1A, ADRA1B, ADRA1D, ADRA2A, ADRB1, ADRB2,...","[148, 147, 146, 150, 153, 154, 155, 1128, 1129...",148|147|146|150|153|154|155|1128|1129|1131|113...,N06AA09,amitriptyline,N06A,Antidepressants,N06AA,Centrally acting sympathomimetics,TCA,"MDD, BPD",NaN,NaN


### repurposing but psych drugs only

In [170]:


drug_to_genes_dict = pd.Series(repurposing_drugs_df_psych.target_entrez_arr.values, index=repurposing_drugs_df_psych.pert_iname.values).to_dict()
save_repurpose_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_target_psych.gmt'
write_gmt(drug_to_genes_dict, save_repurpose_gmt)
read_gmt(save_repurpose_gmt)

,setname,genes_str,genes_arr,num_genes
0,acepromazine,147|148|1806|1812|3315|3355,"[147, 148, 1806, 1812, 3315, 3355]",6
1,acetophenazine,1806|1812,"[1806, 1812]",2
2,agomelatine,3355|3356|3357,"[3355, 3356, 3357]",3
3,amisulpride,1812,[1812],1
4,amitriptyline,10019|11255|1128|1129|1131|1132|1133|122953|14...,"[10019, 11255, 1128, 1129, 1131, 1132, 1133, 1...",35
...,...,...,...,...
86,viloxazine,6524,[6524],1
87,vortioxetine,160728|3315|3350|3355|3357|3358|3360|3361|3362,"[160728, 3315, 3350, 3355, 3357, 3358, 3360, 3...",9
88,ziprasidone,1128|1129|1131|1132|1133|147|148|150|151|152|1...,"[1128, 1129, 1131, 1132, 1133, 147, 148, 150, ...",26
89,zotepine,151|160728|1812|1813|1814|3251|3315|3350|3351|...,"[151, 160728, 1812, 1813, 1814, 3251, 3315, 33...",15


### repurposing but psych drugs cat only

In [171]:
cat_counts = repurposing_drugs_df_psych.ATC_level3_name.value_counts()
display(cat_counts)
drug_categories = list(cat_counts.index)
drug_categories

Antipsychotics         45
Antidepressants        40
Anxiolytics             5
Dopaminergic Agents     1
Name: ATC_level3_name, dtype: int64

['Antipsychotics', 'Antidepressants', 'Anxiolytics', 'Dopaminergic Agents']

In [172]:
cat_to_genelist_psych = defaultdict(set)
for idx, row in repurposing_drugs_df_psych.iterrows():
    genes = row['target_entrez_arr']
    cat  =row['ATC_level3_name']
    cat_to_genelist_psych[cat] = cat_to_genelist_psych[cat].union(genes)


In [173]:
save_repurpose_gmt_cat = '/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_target_psych_cat.gmt'
write_gmt(cat_to_genelist_psych, save_repurpose_gmt_cat)
read_gmt(save_repurpose_gmt_cat)

,setname,genes_str,genes_arr,num_genes
0,Antipsychotics,10599|11255|1128|1129|1131|1132|1133|146|147|1...,"[10599, 11255, 1128, 1129, 1131, 1132, 1133, 1...",70
1,Antidepressants,10019|10599|11255|1128|1129|1131|1132|1133|122...,"[10019, 10599, 11255, 1128, 1129, 1131, 1132, ...",84
2,Anxiolytics,1576|1812|2554|2555|2557|3251|3315|9568,"[1576, 1812, 2554, 2555, 2557, 3251, 3315, 9568]",8
3,Dopaminergic Agents,4057|4067,"[4057, 4067]",2


# 2. expression (cmap) perturbation

In [174]:
cmap_gmt_file_up = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_up_crisp.gmt.gz'
cmap_gmt_file_dn = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_dn_crisp.gmt.gz'

In [133]:
def merge_genes(cmap_df, group_to_setnames_dict):
    # initialize vars
    group_to_genes_dict = defaultdict(set)
    group_to_info_dict = {}
    # make dictionary for easy access
    setname_to_genes_dict = pd.Series(cmap_df.genes_arr.values,cmap_df.setname.values).to_dict()
    # loop through groups
    for group, setnames in group_to_setnames_dict.items():
        # find union gene list
        for setname in setnames:
            group_to_genes_dict[group] = group_to_genes_dict[group].union(set(setname_to_genes_dict[setname]))
        genes_str = '|'.join(sorted(group_to_genes_dict[group]))
        num_genes = len(group_to_genes_dict[group])
        
        group_to_info_dict[group] = {'setnames':setnames,
                                     'genes_str':genes_str, 'genes_arr':sorted(group_to_genes_dict[group]),
                                                                               'num_genes':num_genes}

    return pd.DataFrame.from_dict(group_to_info_dict,orient='index').reset_index()

In [134]:
# print (file_content[0].decode("utf-8") .strip().split('\t'))

In [135]:
%%time
cmap_gmt_up_df = read_gmt(cmap_gmt_file_up)
cmap_gmt_dn_df = read_gmt(cmap_gmt_file_dn)
cmap_gmt_up_df.shape, cmap_gmt_dn_df.shape

CPU times: user 268 ms, sys: 30.9 ms, total: 299 ms
Wall time: 300 ms


((6100, 4), (6100, 4))

In [136]:
cmap_up_drug_group_dict = defaultdict(list) # drug name (without last -####) to setname (original file)
for setname in cmap_gmt_up_df.setname:
    group = '-'.join(setname.split('-')[:-1]).lower()
    cmap_up_drug_group_dict[group].append(setname)
print(len(cmap_up_drug_group_dict), len(cmap_gmt_up_df.setname))

cmap_dn_drug_group_dict = defaultdict(list) # drug name (without last -####) to setname (original file)
for setname in cmap_gmt_dn_df.setname:
    group = '-'.join(setname.split('-')[:-1]).lower()
    cmap_dn_drug_group_dict[group].append(setname)
print(len(cmap_dn_drug_group_dict), len(cmap_gmt_dn_df.setname))


1309 6100
1309 6100


In [138]:
cmap_gmt_up_drug_df = merge_genes(cmap_gmt_up_df, cmap_up_drug_group_dict)
cmap_gmt_dn_drug_df = merge_genes(cmap_gmt_dn_df, cmap_dn_drug_group_dict)
cmap_gmt_up_drug_df.shape, cmap_gmt_dn_drug_df.shape

((1309, 5), (1309, 5))

In [176]:
''

# get all entrez
all_genes_cmap = sorted(set('|'.join(cmap_gmt_up_df.genes_str.values).split('|')))  + sorted(set('|'.join(cmap_gmt_dn_df.genes_str.values).split('|')))
all_genes_cmap  = sorted(set(all_genes_cmap))
gene_to_entrez_cmap_dict = get_entrez_id_dict(all_genes_cmap)



querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-12148...done.
Finished.
186 input query terms found dup hits:
	[('ADAM5', 2), ('ADAMTS2', 2), ('ALDOAP2', 2), ('ANXA2P1', 2), ('ANXA2P2', 2), ('ANXA2P3', 2), ('ARH
555 input query terms found no hit:
	['AARS', 'ACN9', 'ACPP', 'ADCK3', 'ADCK4', 'ADRBK1', 'ADRBK2', 'ADSS', 'AES', 'AIM1', 'AIM1L', 'ALPP
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [180]:
cmap_gmt_up_drug_df['entrez_arr'] = cmap_gmt_up_drug_df.genes_arr.apply(lambda arr: sorted([gene_to_entrez_repurposed_dict.get(x,'') for x in arr]))
cmap_gmt_up_drug_df['entrez'] = cmap_gmt_up_drug_df['entrez_arr'].apply(lambda x: '|'.join(x).strip('|'))

cmap_gmt_dn_drug_df['entrez_arr'] = cmap_gmt_dn_drug_df.genes_arr.apply(lambda arr: sorted([gene_to_entrez_repurposed_dict.get(x,'') for x in arr]))
cmap_gmt_dn_drug_df['entrez'] = cmap_gmt_dn_drug_df['entrez_arr'].apply(lambda x: '|'.join(x).strip('|'))

In [181]:
cmap_gmt_up_drug_df.merge(repurposing_drugs_df,how='inner',left_on='index',right_on='pert_iname').shape

(486, 16)

### 2A. cmap drug perturbation by drug gmt

In [182]:
set_to_genes = pd.Series(cmap_gmt_up_drug_df.entrez_arr.values,index=cmap_gmt_up_drug_df['index'].values).to_dict()
save_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_up_crisp_drug.gmt'
write_gmt(set_to_genes, save_gmt)
read_gmt(save_gmt)

,setname,genes_str,genes_arr,num_genes
0,"1,4-chrysenequinone",1514|1545|2246|2352|2729|3292|3309|3312|3320|3...,"[1514, 1545, 2246, 2352, 2729, 3292, 3309, 331...",19
1,"1,5-isoquinolinediol",10423|11188|1645|22926|32|3710|3770|4886|54884...,"[10423, 11188, 1645, 22926, 32, 3710, 3770, 48...",13
2,"11-deoxy-16,16-dimethylprostaglandin e2",10682|11238|11240|142|1435|1436|1491|1576|1604...,"[10682, 11238, 11240, 142, 1435, 1436, 1491, 1...",36
3,"12,13-eode",1725|316|472|6332,"[1725, 316, 472, 6332]",4
4,"16,16-dimethylprostaglandin e2",11309|1145|1373|1385|1491|1604|2066|210|2261|2...,"[11309, 1145, 1373, 1385, 1491, 1604, 2066, 21...",31
...,...,...,...,...
1304,zidovudine,1103|1132|1141|142|210|2147|2260|2677|27094|29...,"[1103, 1132, 1141, 142, 210, 2147, 2260, 2677,...",25
1305,zimeldine,11240|1436|1545|1559|1812|1910|1969|2046|2051|...,"[11240, 1436, 1545, 1559, 1812, 1910, 1969, 20...",48
1306,zomepirac,10060|1140|116447|1268|1312|134|151|1812|2047|...,"[10060, 1140, 116447, 1268, 1312, 134, 151, 18...",38
1307,zoxazolamine,10013|1021|116444|1277|1312|1384|1543|1545|173...,"[10013, 1021, 116444, 1277, 1312, 1384, 1543, ...",38


In [183]:
set_to_genes = pd.Series(cmap_gmt_dn_drug_df.entrez_arr.values,index=cmap_gmt_dn_drug_df['index'].values).to_dict()
save_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_dn_crisp_drug.gmt'
write_gmt(set_to_genes, save_gmt)
read_gmt(save_gmt)

,setname,genes_str,genes_arr,num_genes
0,"1,4-chrysenequinone",10019|1503|3643|4986|55799,"[10019, 1503, 3643, 4986, 55799]",5
1,"1,5-isoquinolinediol",3309|3312|55869|57105|6347|765|873|9759,"[3309, 3312, 55869, 57105, 6347, 765, 873, 9759]",8
2,"11-deoxy-16,16-dimethylprostaglandin e2",10242|1584|1594|2147|2357|283|29881|3352|3739|...,"[10242, 1584, 1594, 2147, 2357, 283, 29881, 33...",24
3,"12,13-eode",10423|483,"[10423, 483]",2
4,"16,16-dimethylprostaglandin e2",1181|145226|1586|1786|217|231|3781|3787|4688|5...,"[1181, 145226, 1586, 1786, 217, 231, 3781, 378...",18
...,...,...,...,...
1304,zidovudine,1066|143471|153201|1594|2235|2305|2352|2492|29...,"[1066, 143471, 153201, 1594, 2235, 2305, 2352,...",20
1305,zimeldine,10188|1021|1135|1147|1786|1803|3567|5029|51305...,"[10188, 1021, 1135, 1147, 1786, 1803, 3567, 50...",23
1306,zomepirac,10800|10988|1181|2182|2357|2852|36|375519|3781...,"[10800, 10988, 1181, 2182, 2357, 2852, 36, 375...",27
1307,zoxazolamine,1181|1513|1725|2357|2492|2560|2703|28234|2852|...,"[1181, 1513, 1725, 2357, 2492, 2560, 2703, 282...",18


### cmap drug perturbation by drug class gmt

In [184]:
cmap_gmt_up_drug_df_cat = cmap_gmt_up_drug_df.merge(repurposing_drugs_df[['pert_iname','disease_area']],how='inner',
                         left_on='index',right_on='pert_iname')
cmap_gmt_dn_drug_df_cat = cmap_gmt_dn_drug_df.merge(repurposing_drugs_df[['pert_iname','disease_area']],how='inner',
                         left_on='index',right_on='pert_iname')
cmap_gmt_up_drug_df_cat.shape, cmap_gmt_dn_drug_df_cat.shape

((486, 9), (486, 9))

### B. make gmt with drug categories

In [185]:
cat_counts = cmap_gmt_up_drug_df_cat.disease_area.value_counts()[cmap_gmt_up_drug_df_cat.disease_area.value_counts()>10]
display(cat_counts)
drug_categories = list(cat_counts.index)
print(drug_categories)

neurology/psychiatry    109
cardiology               71
infectious disease       44
endocrinology            29
gastroenterology         25
ophthalmology            17
rheumatology             16
dermatology              12
oncology                 12
pulmonary                11
Name: disease_area, dtype: int64

['neurology/psychiatry', 'cardiology', 'infectious disease', 'endocrinology', 'gastroenterology', 'ophthalmology', 'rheumatology', 'dermatology', 'oncology', 'pulmonary']


In [189]:
cat_to_genelist = defaultdict(set)
for idx, row in cmap_gmt_up_drug_df_cat.iterrows():
    genes = row['entrez'].strip('|').split('|')
    for cat in drug_categories:
        if cat in row['disease_area'].split('|'):
            cat_to_genelist[cat] = cat_to_genelist[cat].union(genes)
save_cmap_gmt_cat = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_up_crisp_drug_cat.gmt'
write_gmt(cat_to_genelist, save_cmap_gmt_cat)
read_gmt(save_cmap_gmt_cat)

,setname,genes_str,genes_arr,num_genes
0,infectious disease,10013|10014|10038|10052|10060|1017|1021|10242|...,"[10013, 10014, 10038, 10052, 10060, 1017, 1021...",622
1,cardiology,10013|10014|10038|10052|10060|1017|1021|10280|...,"[10013, 10014, 10038, 10052, 10060, 1017, 1021...",695
2,rheumatology,100|10013|10014|10019|10038|10052|10060|1017|1...,"[100, 10013, 10014, 10019, 10038, 10052, 10060...",563
3,neurology/psychiatry,100|10013|10014|10019|10038|10052|10060|1017|1...,"[100, 10013, 10014, 10019, 10038, 10052, 10060...",770
4,ophthalmology,10013|10038|10060|1017|1021|10242|10280|1036|1...,"[10013, 10038, 10060, 1017, 1021, 10242, 10280...",475
5,endocrinology,100|10013|10014|10019|10038|10052|10060|1017|1...,"[100, 10013, 10014, 10019, 10038, 10052, 10060...",651
6,pulmonary,100|10013|10014|10019|10038|10060|1017|1021|10...,"[100, 10013, 10014, 10019, 10038, 10060, 1017,...",407
7,gastroenterology,10013|10014|10019|10038|10052|10060|1021|10242...,"[10013, 10014, 10019, 10038, 10052, 10060, 102...",602
8,oncology,10013|10014|10019|10038|1017|1021|10280|1036|1...,"[10013, 10014, 10019, 10038, 1017, 1021, 10280...",470
9,dermatology,10013|10014|10019|10038|1017|1021|10242|10280|...,"[10013, 10014, 10019, 10038, 1017, 1021, 10242...",458


In [190]:
cat_counts = cmap_gmt_dn_drug_df_cat.disease_area.value_counts()[cmap_gmt_dn_drug_df_cat.disease_area.value_counts()>10]
display(cat_counts)
drug_categories = list(cat_counts.index)
print(drug_categories)

neurology/psychiatry    109
cardiology               71
infectious disease       44
endocrinology            29
gastroenterology         25
ophthalmology            17
rheumatology             16
dermatology              12
oncology                 12
pulmonary                11
Name: disease_area, dtype: int64

['neurology/psychiatry', 'cardiology', 'infectious disease', 'endocrinology', 'gastroenterology', 'ophthalmology', 'rheumatology', 'dermatology', 'oncology', 'pulmonary']


In [191]:
cat_to_genelist = defaultdict(set)
for idx, row in cmap_gmt_dn_drug_df_cat.iterrows():
    genes = row['entrez'].strip('|').split('|')
    for cat in drug_categories:
        if cat in row['disease_area'].split('|'):
            cat_to_genelist[cat] = cat_to_genelist[cat].union(genes)
save_cmap_gmt_cat = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_dn_crisp_drug_cat.gmt'
write_gmt(cat_to_genelist, save_cmap_gmt_cat)
read_gmt(save_cmap_gmt_cat)

,setname,genes_str,genes_arr,num_genes
0,infectious disease,10013|10188|1019|1021|10242|10280|10376|10423|...,"[10013, 10188, 1019, 1021, 10242, 10280, 10376...",316
1,cardiology,10013|10019|10038|10188|1019|1021|10242|10280|...,"[10013, 10019, 10038, 10188, 1019, 1021, 10242...",359
2,rheumatology,10013|10188|1019|1021|10242|10280|10376|10423|...,"[10013, 10188, 1019, 1021, 10242, 10280, 10376...",305
3,neurology/psychiatry,100|10013|10019|10188|1019|1021|10242|10280|10...,"[100, 10013, 10019, 10188, 1019, 1021, 10242, ...",401
4,ophthalmology,10013|10038|1021|10242|10376|10423|10733|10800...,"[10013, 10038, 1021, 10242, 10376, 10423, 1073...",231
5,endocrinology,10013|10188|1019|1021|10242|10280|10368|10369|...,"[10013, 10188, 1019, 1021, 10242, 10280, 10368...",380
6,pulmonary,10013|10188|1021|10242|1036|10368|10376|1066|1...,"[10013, 10188, 1021, 10242, 1036, 10368, 10376...",252
7,gastroenterology,10013|10019|10188|1021|10242|10280|10376|10423...,"[10013, 10019, 10188, 1021, 10242, 10280, 1037...",307
8,oncology,10013|1021|10242|10280|1036|10369|10376|10423|...,"[10013, 1021, 10242, 10280, 1036, 10369, 10376...",284
9,dermatology,10013|10188|1021|10242|10280|10376|10733|10988...,"[10013, 10188, 1021, 10242, 10280, 10376, 1073...",251
